In [61]:
import os
import pandas as pd

netflix_file = os.path.join('Resources','netflix_titles.csv')
metadata_file = os.path.join('Resources','movies_metadata.csv')
credit_file = os.path.join('Resources','credits.csv')

In [60]:
metadata_df = pd.read_csv(metadata_file, dtype = 'str')

In [43]:
metadata_sub_df = metadata_df[['budget', 'revenue', 'id', 'original_title', 'title']]

In [44]:
metadata_sub_df.head()

,budget,revenue,id,original_title,title
0,30000000,373554033,862,Toy Story,Toy Story
1,65000000,262797249,8844,Jumanji,Jumanji
2,0,0,15602,Grumpier Old Men,Grumpier Old Men
3,16000000,81452156,31357,Waiting to Exhale,Waiting to Exhale
4,0,76578911,11862,Father of the Bride Part II,Father of the Bride Part II


In [45]:
for record in list(metadata_sub_df['budget'].unique()):
    print(record)

30000000
65000000
0
16000000
60000000
58000000
35000000
62000000
44000000
98000000
52000000
16500000
4000000
30250000
50000000
3600000
12000000
18000000
29500000
11000000
10000000
19000000
20000000
33000000
55000000
6000000
15000000
45000000
3500000
8000000
900000
38000000
8169363
47000000
14000000
3000000
7000000
24000000
72000000
1300000
7500000
13365000
25000000
40000000
17000000
31000000
28000000
100000000
2000000
53000000
27000000
90000000
1500000
500000
2
22000000
150000
8
42000000
175000000
2800000
2500000
21000000
27000
700000
5000000
10500000
120
6
5500000
4800000
34000000
400000
32000000
32
6500000
46000000
23000000
11500000
115000000
63000000
45000
26000000
17080000
3200000
13000000
70000000
6900000
57000000
9000000
85000000
13500000
800000
30
48000000
7400000
12500000
200000
1488423
2600000
6244087
9300000
33500000
80000000
220000
275000
25530000
75000000
92000000
1800000
50
1000000
54000000
36000000
2962051
1850000
4500000
2540800
2723903
2479000
325000
2068000
520000
2883

In [46]:
len(metadata_sub_df)

45466

In [47]:
drop_indices = metadata_sub_df['budget'].loc[metadata_sub_df['budget'].str.match('/')].index
clean_metadata_sub_df = metadata_sub_df.drop(drop_indices, axis = 0)

In [51]:
drop_indices

Int64Index([19730, 29503, 35587], dtype='int64')

In [52]:
len(clean_metadata_sub_df)

45463

In [53]:
clean_metadata_sub_df['budget'] = pd.to_numeric(clean_metadata_sub_df['budget'])

In [55]:
clean_metadata_sub_df['revenue'] = pd.to_numeric(clean_metadata_sub_df['revenue'])

In [56]:
clean_metadata_sub_df.dtypes

budget              int64
revenue           float64
id                 object
original_title     object
title              object
dtype: object

In [73]:
output_path = os.path.join('Resources','movie_metadata_transformed.csv')
clean_metadata_sub_df.to_csv(output_path, index = False)

In [74]:
clean_metadata_sub_df.tail()

,budget,revenue,id,original_title,title
45461,0,0.0,439050,رگ خواب,Subdue
45462,0,0.0,111109,Siglo ng Pagluluwal,Century of Birthing
45463,0,0.0,67758,Betrayal,Betrayal
45464,0,0.0,227506,Satana likuyushchiy,Satan Triumphant
45465,0,0.0,461257,Queerama,Queerama


In [66]:
credit_df = pd.read_csv(credit_file)

In [110]:
credit_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [111]:
credit_df.dtypes

cast    object
crew    object
id       int64
dtype: object

In [112]:
test_json = (credit_df['cast'][0])

In [113]:
test_json

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [114]:
import json

In [115]:
test_json = test_json.replace('[', '')
test_json = test_json.replace(']', '')

In [116]:
test_json = test_json.replace('\'', '"')

In [117]:
test_json

'{"cast_id": 14, "character": "Woody (voice)", "credit_id": "52fe4284c3a36847f8024f95", "gender": 2, "id": 31, "name": "Tom Hanks", "order": 0, "profile_path": "/pQFoyx7rp09CJTAb932F2g8Nlho.jpg"}, {"cast_id": 15, "character": "Buzz Lightyear (voice)", "credit_id": "52fe4284c3a36847f8024f99", "gender": 2, "id": 12898, "name": "Tim Allen", "order": 1, "profile_path": "/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg"}, {"cast_id": 16, "character": "Mr. Potato Head (voice)", "credit_id": "52fe4284c3a36847f8024f9d", "gender": 2, "id": 7167, "name": "Don Rickles", "order": 2, "profile_path": "/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg"}, {"cast_id": 17, "character": "Slinky Dog (voice)", "credit_id": "52fe4284c3a36847f8024fa1", "gender": 2, "id": 12899, "name": "Jim Varney", "order": 3, "profile_path": "/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg"}, {"cast_id": 18, "character": "Rex (voice)", "credit_id": "52fe4284c3a36847f8024fa5", "gender": 2, "id": 12900, "name": "Wallace Shawn", "order": 4, "profile_path": "/oGE6JqPP2xH4tN

In [122]:
(eval(test_json))

({'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [123]:
x=eval(test_json)

In [134]:
type(x[0])

dict

In [104]:
`z`a# some JSON:
x =  '{ "name":"John", "age":30, "city":"New York"}'
# parse x:
y = json.loads(x)
# the result is a Python dictionary:
print(y["age"])

30
